In [12]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Importing required libraries

In [13]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import statistics

In [14]:
pd.set_option('display.max_columns', None)

# Merging all datasets

In [15]:
old_path = os.getcwd()

file_names = []

# Iterating through the directories
for folder in os.listdir():
    
    # Iterating through only folders and ignoring hidden files if any
    if os.path.isdir(folder) and not folder.startswith('.'):
        
        # Changing the directory to get the files
        os.chdir(folder)
        
        try:
            # Importing all files
            for file in os.listdir():
                file_names.append(folder + '/' + file)
                
        except:
            print('Error Occured while importing: {} file'.format(file))
            os.chdir(old_path)
        
        # Returning the parent directory
        os.chdir(old_path)

# To store dataframes
dataframes = {}

for file_name in file_names:
    dataframes[file_name.split(".")[0]] = pd.read_csv(os.getcwd() + '/' + file_name, skiprows = 2)
    
dataset = None

# Merging all dataframes excluding few columns to aviod duplicate columns
for df_name, df in dataframes.items():
    columns_to_exclude = ['Year', 'County', 'State', 'Diagnosed Diabetes Percentage']
    columns_to_merge = [col for col in df.columns if col not in columns_to_exclude]

    if dataset is None:
        dataset = df[columns_to_merge]
        
    else:
        dataset = pd.merge(dataset, df[columns_to_merge], on = 'County_FIPS', how = 'inner', suffixes = ('', '_remove'))
    
# To drop last row
dataset = dataset[:-1]

In [16]:
# Changing the directory to grab obesity, inactivity, and diabetes files
os.chdir('/Users/neereshkumarperla/Documents/NeereshMaster/Assignments/MTH 522 - Advance Mathematical Statistics')

obesity = pd.read_excel('cdc-diabetes-2018.xlsx', sheet_name = 'Obesity')
obesity.drop(['YEAR'], axis = 1, inplace = True)

inactivity = pd.read_excel('cdc-diabetes-2018.xlsx', sheet_name = 'Inactivity')
inactivity.rename(columns = {'FIPDS': 'FIPS'}, inplace = True)
inactivity.drop(['YEAR'], axis = 1, inplace = True)

diabetes = pd.read_excel('cdc-diabetes-2018.xlsx', sheet_name = 'Diabetes')
diabetes.drop(['YEAR'], axis = 1, inplace = True)

# Changing the directory to project directory
os.chdir(old_path)

common_column = 'FIPS'

temp_dataset = pd.merge(obesity, inactivity, on=common_column, how='inner', suffixes= ('', '_remove'))
temp_dataset = pd.merge(temp_dataset, diabetes, on=common_column, how='inner', suffixes= ('', '_remove'))

# Removing the duplicate columns
columns_to_remove = [column for column in dataset.columns if '_remove' in column]
dataset.drop(columns_to_remove, axis = 1, inplace = True)

columns_to_remove = [column for column in temp_dataset.columns if '_remove' in column]
temp_dataset.drop(columns_to_remove, axis = 1, inplace = True)
temp_dataset.drop(['STATEW'], axis = 1, inplace = True)

# Renaming the FIPS to County_FIPS to merge the datasets: temp_dataset and dataset
temp_dataset.rename(columns = {'FIPS': 'County_FIPS'}, inplace = True)
temp_dataset['County_FIPS'] = temp_dataset['County_FIPS'].astype('float64')

# This will be our final dataset to perform analysis
final_dataset = pd.merge(temp_dataset, dataset, on = 'County_FIPS', how = 'inner')
final_dataset

,County_FIPS,COUNTY,STATE,% OBESE,% INACTIVE,% DIABETIC,Urban_Rural,Overall SVI,No Health Insurance,Dentists per 100000 Population,Primary Care Physicians per 100000 Population,Number of Primary Care Physicians,Number of Dentists,Commute ≥ 60 min,Single-Parent Households,Overall Minority Status & Language,Unemployed,Group Quarters,No Vehicle,Below Poverty,Civilian with a Disability,Mobile Homes,Aged 17 or Younger,Crowding,Minority,Multi-Unit Structures,Overall Household Composition & Disability,Aged 65 or Older,Income Vulnerability,Overall Socioeconomic Status,Speaks English 'Less than Well',No High School Diploma,Access to Exercise Opportunities,Enrolled in Free or Reduced Lunch,Food Environment Index,Food Insecurity,Limited Access To Healthy Foods,Severe Housing Cost Burden,Children in Poverty,Household with No Internet Service
0,1011.0,Bullock County,Alabama,18.7,17.0,9.4,Rural,0.8898,11.4,20.04812,29.70003,3,2,11.9,0.9955,0.6749,0.9599,0.7689,0.9500,0.9777,0.3983,0.9637,0.2967,0.0000,0.9809,0.3200,0.4438,0.2572,0.8924,0.9771,0.2970,0.9491,1.878321422,75.462645648,4.6,15.7,32.339190387,12.759643917,38.7,30.7
1,2068.0,Denali Borough,Alaska,18.9,16.2,6.8,Rural,0.101,18.5,No Data,No Data,No Data,No Data,6.3,0.0134,0.4540,0.1245,0.9971,0.1210,0.6443,0.3200,0.4390,0.0022,0.6049,0.5492,0.4448,0.0032,0.0022,0.0901,0.1561,0.3582,0.0010,44.359255203,1.2084592145,4.6,15.1,33.09144787,7.2135785007,7.2,6.2
2,2105.0,Hoonah-Angoon Census Area,Alaska,19.4,15.0,7.3,Rural,0.4178,24.7,No Data,No Data,No Data,No Data,2.5,0.4330,0.4575,0.9561,0.8822,0.9946,0.1675,0.5005,0.2569,0.0210,0.8198,0.9124,0.0844,0.2139,0.6428,0.0844,0.3344,0.0000,0.2305,No Data,62.711864407,4.8,14.9,31.130174061,6.432748538,23.2,33.9
3,2195.0,Petersburg Census Area,Alaska,17.2,17.8,9.2,Rural,0.665,9.6,91.01942,183.71096,6,3,0.7,0.5597,0.7657,0.2924,0.5769,0.9844,0.0882,0.7434,0.4830,0.5151,0.7622,0.7711,0.6399,0.8927,0.7947,0.1344,0.1643,0.6558,0.3709,No Data,56.076759062,8.3,12.7,0.6674929034,10.127659574,9.4,15.6
4,2230.0,Skagway Municipality,Alaska,18.3,15.8,6.6,Rural,0.1685,18.2,No Data,No Data,No Data,No Data,0.3,0.2961,0.3919,0.7092,0.9647,0.7685,0.0146,0.1184,0.3916,0.0646,0.8348,0.5043,0.1512,0.0099,0.0423,0.0408,0.1532,0.2970,0.0834,100,2.1428571429,8.2,13.4,0.0599157949,1.5831134565,5,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,51820.0,Waynesboro City,Virginia,19.5,16.6,8.6,Urban,0.6013,13.6,118.72829000000002,97.21607999999999,22,27,3.7,0.8809,0.6587,0.3299,0.0936,0.8201,0.6745,0.4250,0.0831,0.6291,0.3359,0.6224,0.8548,0.8278,0.5215,0.5605,0.5513,0.6310,0.5715,72.779205941,60.857818305,7.5,13.1,7.9537350719,15.561893896,18.9,23.6
350,51830.0,Williamsburg City,Virginia,18.0,15.7,8.5,Urban,0.385,5.9,No Data,13.37435,2,No Data,6.6,0.4330,0.4782,0.7895,0.9936,0.8571,0.8627,0.1038,0.0000,0.0019,0.3359,0.7122,0.9787,0.0204,0.1974,0.2959,0.4997,0.2270,0.0201,99.502416833,41.746411483,7.9,12.3,5.641509498,15.206536541,20,7.9
351,51840.0,Winchester City,Virginia,19.4,16.1,6.9,Urban,0.722,13.8,198.55595999999997,252.86701,71,55,14.4,0.8628,0.8816,0.2188,0.7335,0.8529,0.5127,0.4352,0.0137,0.5715,0.6622,0.7291,0.9058,0.5969,0.2572,0.2694,0.429,0.9287,0.7383,100,63.767565515,7.9,13.1,4.3648085751,12.161515644,18.8,18.7
352,53055.0,San Juan County,Washington,19.3,11.9,4.5,Rural,0.0863,6.4,68.60279,79.62689,14,12,4.7,0.1102,0.4454,0.1245,0.2184,0.2420,0.2704,0.1840,0.2346,0.0080,0.8198,0.3811,0.4317,0.0930,0.9911,0.0274,0.0404,0.5113,0.0150,80.867524891,36.965811966,No Data,9.8,No Data,15.825254941,11,8.6


In [17]:
# Checking if we have any null values after all merging
final_dataset.isnull().any()

County_FIPS                                      False
COUNTY                                           False
STATE                                            False
% OBESE                                          False
% INACTIVE                                       False
% DIABETIC                                       False
Urban_Rural                                      False
Overall SVI                                      False
No Health Insurance                              False
Dentists per 100000 Population                   False
Primary Care Physicians per 100000 Population    False
Number of Primary Care Physicians                False
Number of Dentists                               False
Commute ≥ 60 min                                 False
Single-Parent Households                         False
Overall Minority Status & Language               False
Unemployed                                       False
Group Quarters                                   False
No Vehicle

# Handling Missing Data

## Replacing "No Data" with np.nan

In [18]:
final_dataset.head()

,County_FIPS,COUNTY,STATE,% OBESE,% INACTIVE,% DIABETIC,Urban_Rural,Overall SVI,No Health Insurance,Dentists per 100000 Population,Primary Care Physicians per 100000 Population,Number of Primary Care Physicians,Number of Dentists,Commute ≥ 60 min,Single-Parent Households,Overall Minority Status & Language,Unemployed,Group Quarters,No Vehicle,Below Poverty,Civilian with a Disability,Mobile Homes,Aged 17 or Younger,Crowding,Minority,Multi-Unit Structures,Overall Household Composition & Disability,Aged 65 or Older,Income Vulnerability,Overall Socioeconomic Status,Speaks English 'Less than Well',No High School Diploma,Access to Exercise Opportunities,Enrolled in Free or Reduced Lunch,Food Environment Index,Food Insecurity,Limited Access To Healthy Foods,Severe Housing Cost Burden,Children in Poverty,Household with No Internet Service
0,1011.0,Bullock County,Alabama,18.7,17.0,9.4,Rural,0.8898,11.4,20.04812,29.70003,3,2,11.9,0.9955,0.6749,0.9599,0.7689,0.9500,0.9777,0.3983,0.9637,0.2967,0.0000,0.9809,0.3200,0.4438,0.2572,0.8924,0.9771,0.2970,0.9491,1.878321422,75.462645648,4.6,15.7,32.339190387,12.759643917,38.7,30.7
1,2068.0,Denali Borough,Alaska,18.9,16.2,6.8,Rural,0.101,18.5,No Data,No Data,No Data,No Data,6.3,0.0134,0.4540,0.1245,0.9971,0.1210,0.6443,0.3200,0.4390,0.0022,0.6049,0.5492,0.4448,0.0032,0.0022,0.0901,0.1561,0.3582,0.0010,44.359255203,1.2084592145,4.6,15.1,33.09144787,7.2135785007,7.2,6.2
2,2105.0,Hoonah-Angoon Census Area,Alaska,19.4,15.0,7.3,Rural,0.4178,24.7,No Data,No Data,No Data,No Data,2.5,0.4330,0.4575,0.9561,0.8822,0.9946,0.1675,0.5005,0.2569,0.0210,0.8198,0.9124,0.0844,0.2139,0.6428,0.0844,0.3344,0.0000,0.2305,No Data,62.711864407,4.8,14.9,31.130174061,6.432748538,23.2,33.9
3,2195.0,Petersburg Census Area,Alaska,17.2,17.8,9.2,Rural,0.665,9.6,91.01942,183.71096,6,3,0.7,0.5597,0.7657,0.2924,0.5769,0.9844,0.0882,0.7434,0.4830,0.5151,0.7622,0.7711,0.6399,0.8927,0.7947,0.1344,0.1643,0.6558,0.3709,No Data,56.076759062,8.3,12.7,0.6674929034,10.127659574,9.4,15.6
4,2230.0,Skagway Municipality,Alaska,18.3,15.8,6.6,Rural,0.1685,18.2,No Data,No Data,No Data,No Data,0.3,0.2961,0.3919,0.7092,0.9647,0.7685,0.0146,0.1184,0.3916,0.0646,0.8348,0.5043,0.1512,0.0099,0.0423,0.0408,0.1532,0.2970,0.0834,100,2.1428571429,8.2,13.4,0.0599157949,1.5831134565,5,12.5


In [28]:
# Number of rows that are having "No Data" 
no_data_rows = (final_dataset == "No Data").any(axis = 0)

columns_with_no_data = final_dataset.loc[:, no_data_rows]

columns_with_no_data

,No Health Insurance,Dentists per 100000 Population,Primary Care Physicians per 100000 Population,Number of Primary Care Physicians,Number of Dentists,Commute ≥ 60 min,Access to Exercise Opportunities,Enrolled in Free or Reduced Lunch,Food Environment Index,Limited Access To Healthy Foods,Severe Housing Cost Burden,Children in Poverty
0,11.4,20.04812,29.70003,3,2,11.9,1.878321422,75.462645648,4.6,32.339190387,12.759643917,38.7
1,18.5,No Data,No Data,No Data,No Data,6.3,44.359255203,1.2084592145,4.6,33.09144787,7.2135785007,7.2
2,24.7,No Data,No Data,No Data,No Data,2.5,No Data,62.711864407,4.8,31.130174061,6.432748538,23.2
3,9.6,91.01942,183.71096,6,3,0.7,No Data,56.076759062,8.3,0.6674929034,10.127659574,9.4
4,18.2,No Data,No Data,No Data,No Data,0.3,100,2.1428571429,8.2,0.0599157949,1.5831134565,5
...,...,...,...,...,...,...,...,...,...,...,...,...
349,13.6,118.72829000000002,97.21607999999999,22,27,3.7,72.779205941,60.857818305,7.5,7.9537350719,15.561893896,18.9
350,5.9,No Data,13.37435,2,No Data,6.6,99.502416833,41.746411483,7.9,5.641509498,15.206536541,20
351,13.8,198.55595999999997,252.86701,71,55,14.4,100,63.767565515,7.9,4.3648085751,12.161515644,18.8
352,6.4,68.60279,79.62689,14,12,4.7,80.867524891,36.965811966,No Data,No Data,15.825254941,11


In [25]:
1. replace the colums with "No Data" with np.nan
2. after replacing with nan values re check the values with the above columns whether they are replaced or not.

In [ ]:
# final_dataset = final_dataset.replace("No Data", np.nan)
# final_dataset.isnull().sum()